# Дообучение GigaAM-CTC на FLEURS-Ru


## 1. Установка зависимостей

In [2]:
!pip install torch torchaudio librosa pyannote.audio num2words
!pip install datasets 
!pip install jiwer 
!pip install tqdm pandas soundfile numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
os.chdir('GigaAM')
!pip install -e .
os.chdir('..')

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspaces/speech_course/GigaAM
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for gigaam (pyproject.toml) ... done
  Created wheel for gigaam: filename=gigaam-0.1.0-0.editable-py3-none-any.whl size=7599 sha256=29a79286e09f889cb79cb2aad998ece2fc4039d67c50403006f43b0485fdf933
  Stored in directory: /tmp/pip-ephem-wheel-cache-b76i7qj0/wheels/62/e5/e1/6a5aa813520d3714ad906b66b31ba0f5b7392cb62f13894e97
Successfully built gigaam
  Attempting uninstall: gigaam
    Found existing installation: gigaam 0.1.0
    Uninstalling gigaam-0.1.0:
      Successfully uninstalled gigaam-0.1.0


## 2. Импорты и вспомогательные функции

In [4]:
import pandas as pd
import gigaam
from jiwer import wer, cer
from tqdm import tqdm
import re
from datasets import load_dataset
import tempfile
import soundfile as sf
import numpy as np
import os as os_module
from num2words import num2words

In [5]:
def load_fleurs_data(split='train'):
    import os
    
    dataset_split = 'validation' if split == 'dev' else split
    
    print(f"Загрузка FLEURS (ru_ru, {dataset_split}) из HuggingFace...")

    fleurs_script = 'fleurs/fleurs.py'
    fleurs_backup = 'fleurs/fleurs.py.bak'
    
    renamed = False
    if os.path.exists(fleurs_script):
        try:
            os.rename(fleurs_script, fleurs_backup)
            renamed = True
        except:
            pass
    
    try:
        dataset = load_dataset("google/fleurs", "ru_ru", split=dataset_split)
    finally:
        if renamed and os.path.exists(fleurs_backup):
            try:
                os.rename(fleurs_backup, fleurs_script)
            except:
                pass

    data_list = []
    for item in dataset:
        data_list.append({
            'id': item['id'],
            'audio_array': item['audio']['array'],
            'sampling_rate': item['audio']['sampling_rate'],
            'raw_text': item['raw_transcription'],
            'transcription': item['transcription'],
            'num_samples': item['num_samples'],
            'gender': item['gender']
        })
    
    data = pd.DataFrame(data_list)
    print("Done")
    return data

def convert_numbers_to_text(text):
    pattern = r'\b\d+\b'

    def replace(match):
        number = int(match.group())
        return num2words(number, lang='ru')

    text = re.sub(pattern, replace, text)
    
    return text

def normalize_text(text):

    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text, flags=re.UNICODE)
    text = ' '.join(text.split())
    return convert_numbers_to_text(text)

## 3. Загрузка данных FLEURS

In [6]:
train_data = load_fleurs_data('train')
dev_data = load_fleurs_data('dev')
test_data = load_fleurs_data('test')

print(f"Train: {len(train_data)} samples")
print(f"Validation: {len(dev_data)} samples")
print(f"Test: {len(test_data)} samples")

Загрузка FLEURS (ru_ru, train) из HuggingFace...


/home/vscode/.local/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Done
Загрузка FLEURS (ru_ru, validation) из HuggingFace...


/home/vscode/.local/lib/python3.9/site-packages/datasets/load.py:1429: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Done
Загрузка FLEURS (ru_ru, test) из HuggingFace...
Done
Train: 2562 samples
Validation: 356 samples
Test: 775 samples


In [7]:
print("Пример записи из train:")
sample = train_data.iloc[1]
print(f"Оригинальный текст: {sample['raw_text']}")
print(f"Нормализованный текст: {normalize_text(sample['raw_text'])}")

Пример записи из train:
Оригинальный текст: На 3-м месте — Хэмлин, который отстает на двадцать очков, но опережает на пять Бойера. Кейн и Трукс-младший занимают 5-е и 6-е места соответственно, набрав по 2220 и 2207 очков.
Нормализованный текст: на 3м месте хэмлин который отстает на двадцать очков но опережает на пять бойера кейн и труксмладший занимают 5е и 6е места соответственно набрав по две тысячи двести двадцать и две тысячи двести семь очков


## 4. Загрузка модели GigaAM

In [8]:
model = gigaam.load_model("ctc")

/workspaces/speech_course/GigaAM/gigaam/__init__.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location="cpu")


## 5. Тестирование на одном образце

In [9]:
sample = dev_data.iloc[0]
reference = normalize_text(sample['raw_text'])

print("Тестирование модели на одном образце...")
print(f"ID: {sample['id']}")

with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_file:
    tmp_path = tmp_file.name
    sf.write(tmp_path, sample['audio_array'], sample['sampling_rate'])

prediction = model.transcribe(tmp_path)
prediction_normalized = normalize_text(prediction)

os_module.unlink(tmp_path)

print(f"\nReference:  {reference}")
print(f"Prediction: {prediction_normalized}")
print(f"\nСовпадение: {reference == prediction_normalized}")

Тестирование модели на одном образце...
ID: 1614


/workspaces/speech_course/GigaAM/gigaam/preprocess.py:40: UserWarning: The given buffer is not writable, and PyTorch does not support non-writable tensors. This means you can write to the underlying (supposedly non-writable) buffer using the tensor. You may want to copy the buffer to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:1560.)
  return torch.frombuffer(audio, dtype=torch.int16).float() / 32768.0



Reference:  они умеют отлично видеть в темноте при помощи ночного видения и почти незаметно передвигаться оцелоты выслеживают добычу сливаясь с окружающей обстановкой а затем набрасываются на добычу
Prediction: они умеют отлично видеть в темноте при помощи ночного видения и почти незаметно передвигаться а цилоты выслеживают добычу сливаясь с окружающей обстановкой а затем набрасываются на добычу

Совпадение: False


## 6. Инференс на валидационном наборе

In [10]:
def run_inference(model, data_df):
    predictions = []
    references = []
    
    for idx, row in tqdm(data_df.iterrows(), total=len(data_df), desc="Inference"):
        try:
            with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp_file:
                tmp_path = tmp_file.name
                sf.write(tmp_path, row['audio_array'], row['sampling_rate'])
            
            prediction = model.transcribe_longform(tmp_path)
            prediction = normalize_text(prediction[0]['transcription'])
            
            os_module.unlink(tmp_path)
            
            reference = normalize_text(row['raw_text'])
            predictions.append(prediction)
            references.append(reference)
        except Exception as e:
            print("Error")
            predictions.append("")
            references.append(normalize_text(row['raw_text']))
            if os_module.path.exists(tmp_path):
                os_module.unlink(tmp_path)
    
    return predictions, references

In [ ]:
# os.environ['HF_TOKEN']
predictions, references = run_inference(model, dev_data)

results_df = pd.DataFrame({
    'audio_id': dev_data['id'].values,
    'reference': references,
    'prediction': predictions
})

results_df.to_csv('dev_predictions.csv', index=False)

Inference:   0%|          | 0/356 [00:00<?, ?it/s]DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
/home/vscode/.local/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
INFO:pytorch_light

Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.


Inference: 100%|██████████| 356/356 [09:36<00:00,  1.62s/it]


## 7. Расчет метрик WER и CER

In [12]:
valid_pairs = [(ref, pred) for ref, pred in zip(references, predictions) 
               if pred and ref]

if valid_pairs:
    references_valid, predictions_valid = zip(*valid_pairs)

    wer_score = wer(list(references_valid), list(predictions_valid))
    cer_score = cer(list(references_valid), list(predictions_valid))
    
    print("РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print(f"Всего образцов: {len(dev_data)}")
    print(f"Валидных предсказаний: {len(valid_pairs)}")
    print(f"Word Error Rate (WER):      {wer_score*100:.2f}%")
    print(f"Character Error Rate (CER): {cer_score*100:.2f}%")

РЕЗУЛЬТАТЫ ОЦЕНКИ
Всего образцов: 356
Валидных предсказаний: 356
Word Error Rate (WER):      7.97%
Character Error Rate (CER): 3.24%


## 8. Анализ результатов

In [15]:
print("Примеры ПРАВИЛЬНЫХ предсказаний:")

correct_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref == pred and correct_count < 3:
        print(f"\n[Пример {correct_count + 1}]")
        print(f"Text: {ref}")
        correct_count += 1

print(f"\nВсего точных совпадений: {sum(1 for r, p in zip(references_valid, predictions_valid) if r == p)}")

Примеры ПРАВИЛЬНЫХ предсказаний:

[Пример 1]
Text: о первых случаях заболевания в этом сезоне было сообщено в июле

[Пример 2]
Text: изложенные мнения часто поверхностны расплывчаты и чрезмерно упрощены по сравнению с повсеместно доступной более подробной информацией

[Пример 3]
Text: среди примеров активного отдыха на объекте охота рыбная ловля фотографирование наблюдение за птицами посещение парков и изучение информации об экосистеме

Всего точных совпадений: 175


In [17]:
print("\nПримеры предсказаний С ОШИБКАМИ:")

error_count = 0
for i, (ref, pred) in enumerate(zip(references_valid, predictions_valid)):
    if ref != pred and error_count < 3:
        print(f"\n[Пример {error_count + 1}]")
        print(f"Reference:  {ref}")
        print(f"Prediction: {pred}")
        error_count += 1

print(f"\nВсего не точных совпадений: {sum(1 for r, p in zip(references_valid, predictions_valid) if r != p)}")


Примеры предсказаний С ОШИБКАМИ:

[Пример 1]
Reference:  они умеют отлично видеть в темноте при помощи ночного видения и почти незаметно передвигаться оцелоты выслеживают добычу сливаясь с окружающей обстановкой а затем набрасываются на добычу
Prediction: они умеют отлично видеть в темноте при помощи ночного видения и почти незаметно передвигаться ацилоты выслеживают добычу сливаясь с окружающей обстановкой а затем набрасываются на добычу

[Пример 2]
Reference:  он сказал что создал дверной звонок работающий от wifi
Prediction: он сказал что создал дверной звонок работающий от вай фай

[Пример 3]
Reference:  в японии приблизительно семь тысяч островов самый большой из которых хонсю что делает японию 7м по величине островом в мире
Prediction: в японии приблизительно семь тысяч островов самый большой из которых хонти что делает японию седьмым по величине островом в мире

Всего не точных совпадений: 181
